In [2]:
#if you still need to install beautifulsoup4

! pip install requests beautifulsoup4 lxml

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Basis-URL
base_url = "https://www.ots.at/suche"

# Angepasste Anfrage-Parameter
params = {
    "query": "",
    "seite": 1,
    "emittentId": 82,  # Angepasste Emittent-ID
    "startDate": 1482015601,  # Angepasster Startzeitpunkt (Unix-Zeitstempel)
    "endDate": 1545173999,    # Angepasster Endzeitpunkt (Unix-Zeitstempel)
    "channel": "index",
    "attachment": ""
}

# Funktion zum Abrufen von Artikeldetails
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Fehler beim Abrufen der Artikel-URL: {article_url}")
            return None, None, None

        soup = BeautifulSoup(response.text, "html.parser")
        # Titel
        title = soup.find("h1").get_text(strip=True)
        # Inhalt
        content_div = soup.find("div", {"itemprop": "articleBody"})
        content = content_div.get_text(strip=True) if content_div else "Kein Inhalt verfügbar"
        # Veröffentlichungsdatum
        date_meta = soup.find("meta", itemprop="datePublished")
        date_time = soup.find("time", {"itemprop": "datePublished"})
        date = (
            date_meta["content"] if date_meta and date_meta.has_attr("content") 
            else date_time["datetime"] if date_time and date_time.has_attr("datetime") 
            else date_time.get_text(strip=True) if date_time 
            else "Kein Datum verfügbar"
        )
        return title, content, date
    except Exception as e:
        print(f"Fehler beim Abrufen der Details: {e}")
        return None, None, None

# Funktion zum Scrapen einer Seite
def scrape_page(page_number):
    params["seite"] = page_number
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Fehler beim Abrufen der Seite {page_number}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("mat-card")
    data = []

    for article in articles:
        try:
            # Titel
            title_tag = article.find("h1", class_="display-3")
            title = title_tag.get_text(strip=True) if title_tag else "Kein Titel"

            # Teaser
            teaser_tag = article.find("p", class_="lead")
            teaser = teaser_tag.get_text(strip=True) if teaser_tag else "Kein Teaser"

            # Link
            link_tag = article.find("a", class_="link-detailed-view")
            link = f"https://www.ots.at{link_tag['href']}" if link_tag else "Kein Link"

            # Details scrapen
            article_title, article_content, article_date = scrape_article_content(link)

            # Daten hinzufügen
            data.append({
                "Titel": title,
                "Teaser": teaser,
                "Datum": article_date,
                "Link": link,
                "Inhalt": article_content
            })

        except Exception as e:
            print(f"Fehler beim Verarbeiten eines Artikels: {e}")
            continue

    return data

# Hauptfunktion für mehrere Seiten
def scrape_all_pages(start_page=1, end_page=5):
    all_data = []
    for page in range(start_page, end_page + 1):
        print(f"Scraping Seite {page}...")
        page_data = scrape_page(page)
        all_data.extend(page_data)
        time.sleep(2)  # Wartezeit zwischen Anfragen

    return all_data

# Ergebnisse speichern
def save_to_csv(data, filename="ots_scraper_data_fpoe_partei.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Daten erfolgreich gespeichert in {filename}")

# Ausführung
if __name__ == "__main__":
    scraped_data = scrape_all_pages(start_page=1, end_page=6)  
    if scraped_data:
        save_to_csv(scraped_data)
    else:
        print("Keine Daten gescrapt.")


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Daten erfolgreich gespeichert in ots_scraper_data_fpoe_partei.csv


In [6]:
df_fpoe = pd.read_csv("ots_scraper_data_fpoe_partei.csv")

display(df_fpoe)

,Titel,Teaser,Datum,Link,Inhalt
0,Vizekanzler Strache kritisiert Gründung einer ...,Wien (OTS) - Österreichs Vizekanzler Heinz-Chr...,2018-12-14T14:04:03+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Wien (OTS) - Österreichs Vizekanzler Heinz-Chr...
1,Hafenecker/Nehammer: „Notstandshilfe wird als ...,Wien (OTS) - Entgegen anderslautender heutiger...,2018-12-04T11:54:16+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Wien (OTS) - Entgegen anderslautender heutiger...
2,Freiheitliche Jugend: Bundesseminar „Zeichen s...,150 Jugendliche trafen sich in Oberösterreich ...,2018-12-04T09:20:08+01:00,https://www.ots.at/presseaussendung/OTS_201812...,150 Jugendliche trafen sich in Oberösterreich ...
3,Philippa Strache: „Neues Wiener Tierhaltegeset...,„Wiener Regelung (wenn man sie so bezeichnen m...,2018-11-30T14:27:14+01:00,https://www.ots.at/presseaussendung/OTS_201811...,„Wiener Regelung (wenn man sie so bezeichnen m...
4,FPÖ: AVISO: Buchpräsentation morgen in Graz „H...,WIEN (OTS) - Hiermit laden wir die Kolleginnen...,2018-11-28T18:02:59+01:00,https://www.ots.at/presseaussendung/OTS_201811...,WIEN (OTS) - Hiermit laden wir die Kolleginnen...
5,"FPÖ/ÖSR-Neubauer: ""LAbg. Dr. Walter Ratt seit ...","""Silbernes Ehrenzeichen für Verdienste um die ...",2018-11-28T10:33:11+01:00,https://www.ots.at/presseaussendung/OTS_201811...,"""Silbernes Ehrenzeichen für Verdienste um die ..."
6,Vizekanzler Strache: Kosovo muss Zollerhöhung ...,Wien (OTS) - Empört zeigte sich Österreichs Vi...,2018-11-23T12:13:05+01:00,https://www.ots.at/presseaussendung/OTS_201811...,Wien (OTS) - Empört zeigte sich Österreichs Vi...
7,FPÖ: „Das neue Waffengesetz ist auf den Weg ge...,„Neben der Umsetzung der EU-Waffenrechtsrichtl...,2018-11-20T08:41:44+01:00,https://www.ots.at/presseaussendung/OTS_201811...,„Neben der Umsetzung der EU-Waffenrechtsrichtl...
8,"FPÖ/FJ-Raml: ""Streiks in der Metalltechnischen...","""SJ-Herr bedient sich Klassenkampfrhetorik und...",2018-11-13T09:30:22+01:00,https://www.ots.at/presseaussendung/OTS_201811...,"""SJ-Herr bedient sich Klassenkampfrhetorik und..."
9,Freiheitliche Jugend feiert mit Vizekanzler HC...,60 Jahre im Zeichen von Patriotismus und Gesch...,2018-10-28T10:32:50+01:00,https://www.ots.at/presseaussendung/OTS_201810...,60 Jahre im Zeichen von Patriotismus und Gesch...
